# Analysis

In [1]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [2]:
# set up plotting settings for dark mode.
from jupyterthemes import jtplot
jtplot.style(theme="grade3", context="notebook", ticks=True, grid=False)
from src.plot_utils import ps_defaults
ps_defaults(use_tex=False, dpi=150)

In [6]:
from emukit.test_functions import branin_function
from emukit.core import ParameterSpace, ContinuousParameter

f, _ = branin_function()
parameter_space = ParameterSpace([ContinuousParameter('x1', -5, 10),
                                  ContinuousParameter('x2', 0, 15)])

In [8]:
from emukit.core.initial_designs.latin_design import LatinDesign

design = LatinDesign(parameter_space) 
num_data_points = 20
X = design.get_samples(num_data_points)

In [12]:
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper

Y = f(X)
model_gpy = GPRegression(X,Y)
model_emukit = GPyModelWrapper(model_gpy)

[autoreload of six failed: Traceback (most recent call last):
  File "/home/users/sithom/seager19/env/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/users/sithom/seager19/env/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/users/sithom/seager19/env/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/users/sithom/seager19/env/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 287, in update_class
    old_obj = getattr(old, key)
  File "/home/users/sithom/seager19/env/lib/python3.8/site-packages/six.py", line 98, in __get__
    setattr(obj, self.name, result)  # Invokes __set__.
AttributeError: 'NoneType' object has no attribute 'cStringIO'
]
[autoreload of pkg_resources failed: Traceback (most recent call last):
  File 

AttributeError: module 'numpy.polynomial' has no attribute 'hermite'

In [5]:
!pip freeze

appdirs==1.4.4
asciitree==0.3.3
astroid==2.9.0
bokeh==2.3.3
brotlipy==0.7.0
certifi==2020.6.20
cffi @ file:///tmp/build/80754af9/cffi_1605538068321/work
cftime==1.5.0
chardet @ file:///tmp/build/80754af9/chardet_1605303185383/work
click==8.0.1
cloudpickle==1.6.0
conda==4.9.2
conda-package-handling @ file:///tmp/build/80754af9/conda-package-handling_1603018141399/work
configparser==5.0.2
cryptography @ file:///tmp/build/80754af9/cryptography_1605544487601/work
cycler==0.10.0
Cython==0.29.26
dask==2021.6.2
decorator==5.1.1
distributed==2021.6.2
docker-pycreds==0.4.0
emcee==3.1.1
entrypoints==0.3
fasteners==0.16.3
fastprogress==1.0.0
fsspec==2021.6.1
gitdb==4.0.7
GitPython==3.1.14
h5netcdf==0.11.0
h5py==3.3.0
HeapDict==1.0.1
idna @ file:///tmp/build/80754af9/idna_1593446292537/work
intake==0.6.2
intake-esm==2021.1.15
isort==5.10.1
Jinja2==3.0.1
kiwisolver==1.3.1
lazy-object-proxy==1.6.0
llvmlite==0.37.0
locket==0.2.1
MarkupSafe==2.0.1
matplotlib==3.4.2
mccabe==0.6.1
msgpack==1.0.2
nc-time

In [ ]:


pip install --upgrade matplotlib
conda remove numpy
conda install numpy=1.13
